In [1]:
import glob
import json
import pickle

총 50개의 평가용 테스트 케이스 확보 예정
- 많으면 많을 수록 좋지만 평가 비용(돈, 시간 등) 커지는 이슈 존재
- 아래 형식의 평가 데이터 리턴하는 함수 제작하는 것이 이번 노트북의 목표

In [1]:
def get_eval_data():
    return None

2명의 대화 - 20개

In [2]:
conversations = []

paths = glob.glob('./res/297.SNS 데이터 고도화/01-1.정식개방데이터/Validation/02.라벨링데이터/VL/*json')
target_count = 10
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] == 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    print('Repeated Conversations')
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

count

10

3명 이상의 대화 - 30개

In [3]:
target_count = 10
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] > 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

count

10

3천자 이상의 대화로 변환
- 데이터셋에는 3천자를 넘는 데이터가 단 한개도 존재하지 않음
- Anthropic Claude 3.5 Sonnet을 사용해서 대화를 주고 앞에 내용을 추측한 내용으로 대화 앞에 붙여서 3천자 이상으로 변환
- 변환 Prompt 및 코드는 data_long.ipynb 참고

In [4]:
with open('./res/conv_long.pickle', 'rb') as f:
    conv_long = pickle.load(f)

conv_long

['먼저 P01은 개인 PT를 받고 싶지만 트레이너가 자신에게 너무 집중하는 것이 부담스럽다고 말합니다. 그래서 다른 방법을 찾아보고 싶어 하는 것 같네요. P02는 5대 1 PT를 추천하고, P03는 그런 방식의 PT가 있는지 궁금해 합니다. P01은 필라테스 5대 1 수업을 해봤다고 말하고, P02는 피티에도 그런 방식이 있을 것 같다고 말합니다.',
 '그 후 P03은 필라테스가 좋은지 물어보고, P01은 필라테스가 헬스와 비슷하게 곡소리가 나고 평소 쓰지 않던 근육을 사용해서 느낌이 좋다고 말합니다. P02는 필라테스를 하면 유연성이 많이 늘었을 것 같다고 말하고, P03은 원래 유연했던 사람은 더 유연해질 게 없을 것 같다고 말합니다.',
 'P01은 필라테스에서 허벅지와 엉덩이 근육을 많이 사용한다고 하고, P02는 자신이 뻣뻣해서 필라테스를 하다가 죽을 것 같다고 말합니다. P03은 자신도 필라테스를 하면 곡소리가 날 것 같다고 말합니다.',
 '그 후 P01은 다들 어떤 운동을 좋아하는지 물어보고, P02는 공으로 하는 운동을 좋아한다고 말합니다. P03은 풋살을 하는 이유가 그런 것 같다고 말하고, 자신은 춤을 배우고 싶다고 말합니다. P01은 춤 학원이 많이 있을 것 같다고 말하고, P03은 집 주변에는 없어서 시내에 가야 한다고 말합니다.',
 '마지막으로 P01은 나중에 여유가 생기면 P03과 함께 보컬 학원에 다니고 싶다고 말하고, P02는 둘이 같이 다니면 좋을 것 같다고 말합니다. P03도 보컬 학원이 재미있을 것 같다고 말하며, 노래를 잘 부르고 싶다고 합니다.']

In [5]:
conversations[-2] = '\n'.join(conv_long) + '\n' + conversations[-2]

In [6]:
with open('./res/eval_data.pickle', 'wb') as f:
    pickle.dump(conversations, f)

In [7]:
def get_eval_data():
    with open('./res/eval_data.pickle', 'rb') as f:
        eval_data = pickle.load(f)

    return eval_data

위 함수를 eval.py에 저장